In [5]:
import pandas as pd

vehicle_df = pd.read_csv("../../datasets/vehicle.csv")

In [20]:
import pandas as pd

# 1) Load
veh = pd.read_csv("../../datasets/vehicle.csv")

# 2) Drop unwanted cols
veh.drop(
    columns=[
        "VEHICLE_ID",
        "LEVEL_OF_DAMAGE",
        "TOWED_AWAY_FLAG",
        "CAUGHT_FIRE",
        "ROAD_SURFACE_TYPE_DESC",
        "TRAFFIC_CONTROL_DESC",
        "VEHICLE_MAKE",
        "VEHICLE_MODEL",
        "VEHICLE_COLOUR_2",
        "REG_STATE",
        "VEHICLE_POWER",
        "VEHICLE_WEIGHT",
        "VEHICLE_YEAR_MANUF",  # Can keep this later when process properly
    ],
    errors="ignore",
    inplace=True,
)

# 3) Toss the  'G' = “unknown what is being towed” rows:
veh = veh[veh["TRAILER_TYPE"] != "G"]

# 4) Single boolean flag: H means “no trailer”, everything else (A–L, I, J, K, etc.) → tow
veh["HAS_TRAILER"] = (veh["TRAILER_TYPE"] != "H").astype(int)

# 5) Vehicle count for aggregation
veh["VEHICLE_COUNT"] = 1

# 6) Per-accident aggregation:
#    - HAS_TRAILER → max (0 if none, 1 if any)
#    - VEHICLE_COUNT → sum
veh_sum = veh.groupby("ACCIDENT_NO", as_index=False).agg(
    ANY_TRAILER=("HAS_TRAILER", "max"),
    NUM_VEHICLES=("VEHICLE_COUNT", "sum"),
)

veh_sum

,ACCIDENT_NO,ANY_TRAILER,NUM_VEHICLES
0,T20120000009,0,1
1,T20120000012,0,2
2,T20120000013,0,2
3,T20120000018,0,1
4,T20120000021,0,1
...,...,...,...
178638,T20250000840,0,1
178639,T20250001075,0,2
178640,T20250001671,0,2
178641,T20250001949,0,1


In [26]:
veh_sum["NUM_VEHICLES"].value_counts()

NUM_VEHICLES
2     100458
1      58367
3      14982
4       3571
5        890
6        234
7         80
8         29
9         10
10         9
12         4
11         3
13         2
16         1
14         1
21         1
19         1
Name: count, dtype: int64